In [1]:
import pymongo
from pymongo.errors import BulkWriteError

In [2]:
#Check mongodb is running in the terminal
db = pymongo.MongoClient().doc_store

# In order to reset database
#db.vocab.drop()

In [3]:
#Make a generator to call document level word counts one at a time from mongo

def wordcount_generator():
    full_corpus = db.count.find({})

    for record in full_corpus:
        yield record['count']

In [ ]:
from collections import Counter
    
c = Counter()

wordcount = wordcount_generator()

print("Counting Words")
i = 0
for w_c in wordcount:
    if i % 250000 == 0:
        print("Counting Words "+str(i))
        
    i = i + 1
    c.update(w_c)
    
# This creates a counter with 66,634,259 unique entries

Counting Words
Counting Words 0
Counting Words 250000
Counting Words 500000
Counting Words 750000
Counting Words 1000000
Counting Words 1250000


In [ ]:
print("Inserting Vocab")
error_log = []

i = 0
for word in c.items():
    if i % 1000000 == 0:
        print("Inserting Vocab " + str(i))
        
    i = i + 1
    try:
        db.vocab.insert_one({'_id': word[0], 'count': word[1]})
    except Exception as e:
        error_log.append(e)
        continue


In [ ]:
print("There were "+str(len(c))+" records to insert. There were "+str(len(error_log))+" errors. Theere were "+str(db.vocab.count())+" records inserted.")
print("All records accounted for.")

In [ ]:
from collections import Counter
    
c2 = Counter()

wordcount = wordcount_generator()

print("Counting Words")
i = 0
for w_c in wordcount:
    if i % 250000 == 0:
        print("Counting Words "+str(i))
        
    i = i + 1
    c2.update(w_c.keys())
    
# This creates a counter with 66,634,259 unique entries

In [ ]:
print("Inserting Vocab")
error_log = []

i = 0
for word in c.items():
    if i % 1000000 == 0:
        print("Inserting Vocab " + str(i))
        
    i = i + 1
    try:
        db.vocab.update({"_id":  word[0]},{"$set" : {'docs': word[1]}})
    except Exception as e:
        error_log.append(e)
        continue